In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# Python 3.12
%env CLEARML_API_ACCESS_KEY=17EH8MKOUXFTCCY4ZNGB7KOEIGXU59
%env CLEARML_API_SECRET_KEY=I3XdWFGE-rw1-16wvP-F1uV6Uq143N7igmA6hAOetL8a8oj-4Xi32kYHC78lhjSCRzc

In [1]:
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_community.llms import GigaChat
from pydantic import BaseModel, Field
from typing import Optional
from clearml import Task, Logger, StorageManager, Dataset
import os
import pickle
import requests

In [8]:
class EssayEvaluation(BaseModel):
    H1: int = Field(description="Балл по критерию K1", ge=0, le=1)
    H1_explanation: str = Field(description="Обоснование оценки по критерию K1")
    H2: int = Field(description="Балл по критерию K2", ge=0, le=3)
    H2_explanation: str = Field(description="Обоснование оценки по критерию K2")
    H3: int = Field(description="Балл по критерию K3", ge=0, le=2)
    H3_explanation: str = Field(description="Обоснование оценки по критерию K3")
    H4: int = Field(description="Балл по критерию K4", ge=0, le=1)
    H4_explanation: str = Field(description="Обоснование оценки по критерию K4")

def setup_llm():
    """Настройка GigaChat"""
    llm = GigaChat(
        credentials="YzJiZGZmM2MtYjdlMy00OGE5LWE5YmItNTg3MDZiYjJiZTE3OmY2OTc2NmJlLWU2NmQtNDBhYS1iNjVhLWFkZDQxNmNkYmFiMA==", 
        verify_ssl_certs=False,
        model='GigaChat-2',
        temperature=0.1,
    )
    return llm

In [9]:
def create_prompts(parser):
    """Создание промптов для разных типов сочинений"""
    
    # Промпт для типа 1 (13.1 - лингвистическое сочинение)
    prompt_type1 = ChatPromptTemplate.from_messages([
        ("system", """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
Оцени сочинение по критериям К1-К4 для задания типа 13.1 (лингвистическое сочинение).

КРИТЕРИИ ОЦЕНКИ для типа 1:
К1 (0-1 балл): Наличие обоснованного ответа на лингвистический вопрос. Рассуждение на теоретическом уровне, отсутствие фактических ошибок.
К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, соответствующих теме.
К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.

Текст задания: {task_text}
Тип сочинения: 1 (13.1)

Верни ответ строго в указанном JSON-формате."""),
        ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())

    # Промпт для типа 2 (13.2 - литературно-тематическое сочинение)
    prompt_type2 = ChatPromptTemplate.from_messages([
        ("system", """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
Оцени сочинение по критериям К1-К4 для задания типа 13.2 (литературно-тематическое сочинение).

КРИТЕРИИ ОЦЕНКИ для типа 2:
К1 (0-1 балл): Понимание смысла фрагмента текста - верное объяснение содержания фрагмента.
К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, иллюстрирующих понимание фрагмента.
К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.

Текст задания: {task_text}
Тип сочинения: 2 (13.2)

Верни ответ строго в указанном JSON-формате."""),
        ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())

    # Промпт для типа 3 (13.3 - морально-нравственное сочинение)
    prompt_type3 = ChatPromptTemplate.from_messages([
        ("system", """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
Оцени сочинение по критериям К1-К4 для задания типа 13.3 (морально-нравственное сочинение).

КРИТЕРИИ ОЦЕНКИ для типа 3:
К1 (0-1 балл): Наличие обоснованного ответа на поставленный вопрос.
К2 (0-3 балла): Приведение одного примера из опорного текста и одного примера из личного опыта/литературы.
К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.

Текст задания: {task_text}
Тип сочинения: 3 (13.3)

Верни ответ строго в указанном JSON-формате."""),
        ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())

    return prompt_type1, prompt_type2, prompt_type3

In [10]:
def evaluate_essays(test_csv_path, output_csv_path="submission.csv"):
    """Основная функция для оценки сочинений"""
    
    # Инициализация компонентов
    llm = setup_llm()
    parser = PydanticOutputParser(pydantic_object=EssayEvaluation)
    prompt_type1, prompt_type2, prompt_type3 = create_prompts(parser)
    
    # Создаем цепочки для каждого типа сочинения
    chain1 = LLMChain(llm=llm, prompt=prompt_type1, output_parser=parser)
    chain2 = LLMChain(llm=llm, prompt=prompt_type2, output_parser=parser)
    chain3 = LLMChain(llm=llm, prompt=prompt_type3, output_parser=parser)
    
    # Загрузка тестовых данных
    test_data = pd.read_csv(test_csv_path)
    results = []
    
    print(f"Начинаем обработку {len(test_data)} сочинений...")
    
    for idx, row in test_data.iterrows():
        try:
            essay_type = row["essay_type"]
            essay_text = row["essay_text"]
            task_text = row["task_text"]
            
            # Получаем предсказание в зависимости от типа сочинения
            if essay_type == 1:
                result = chain1.invoke({"essay_text": essay_text, "task_text": task_text})
            elif essay_type == 2:
                result = chain2.invoke({"essay_text": essay_text, "task_text": task_text})
            elif essay_type == 3:
                result = chain3.invoke({"essay_text": essay_text, "task_text": task_text})
            else:
                print(f"Неизвестный тип сочинения: {essay_type}")
                continue
                
            evaluation = result["text"]
            
            results.append({
                "essay_id": row["essay_id"],
                "H1": evaluation.H1,
                "H1_explanation": evaluation.H1_explanation,
                "H2": evaluation.H2,
                "H2_explanation": evaluation.H2_explanation,
                "H3": evaluation.H3,
                "H3_explanation": evaluation.H3_explanation,
                "H4": evaluation.H4,
                "H4_explanation": evaluation.H4_explanation
            })
            
            print(f"Обработано сочинение {idx + 1}/{len(test_data)} (ID: {row['essay_id']})")
            
        except Exception as e:
            print(f"Ошибка при обработке essay_id {row['essay_id']}: {str(e)}")
            # Добавляем запись с нулевыми значениями в случае ошибки
            results.append({
                "essay_id": row["essay_id"],
                "H1": 0,
                "H1_explanation": f"Ошибка обработки: {str(e)}",
                "H2": 0,
                "H2_explanation": f"Ошибка обработки: {str(e)}",
                "H3": 0,
                "H3_explanation": f"Ошибка обработки: {str(e)}",
                "H4": 0,
                "H4_explanation": f"Ошибка обработки: {str(e)}"
            })
    
    # Сохраняем результаты
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)
    print(f"Результаты сохранены в {output_csv_path}")
    
    return results_df

In [12]:
def analyze_results(results_df):
    """Анализ и вывод статистики результатов"""
    print("\n" + "="*50)
    print("СТАТИСТИКА ОЦЕНОК:")
    print("="*50)
    print(f"Средний балл H1: {results_df['H1'].mean():.2f}")
    print(f"Средний балл H2: {results_df['H2'].mean():.2f}")
    print(f"Средний балл H3: {results_df['H3'].mean():.2f}")
    print(f"Средний балл H4: {results_df['H4'].mean():.2f}")
    
    print(f"\nОбщее количество обработанных сочинений: {len(results_df)}")
    print(f"Сочинений с ошибками обработки: {len(results_df[results_df['H1_explanation'].str.startswith('Ошибка обработки')])}")
    
    # Показываем первые несколько результатов
    print("\n" + "="*50)
    print("ПЕРВЫЕ 3 РЕЗУЛЬТАТА:")
    print("="*50)
    for i in range(min(3, len(results_df))):
        print(f"\nСочинение ID: {results_df.iloc[i]['essay_id']}")
        print(f"  H1: {results_df.iloc[i]['H1']} - {results_df.iloc[i]['H1_explanation'][:100]}...")
        print(f"  H2: {results_df.iloc[i]['H2']} - {results_df.iloc[i]['H2_explanation'][:100]}...")

if __name__ == "__main__":
    # Основной блок выполнения
    test_csv_path = "D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/essays.csv"  # Укажите путь к вашему test.csv
    
    try:
        # Оцениваем сочинения
        results_df = evaluate_essays(test_csv_path)
        
        # Анализируем результаты
        analyze_results(results_df)
        
        print("\nОбработка завершена успешно!")
        
    except FileNotFoundError:
        print(f"Ошибка: Файл {test_csv_path} не найден.")
        print("Убедитесь, что файл test.csv находится в правильной директории.")
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

Начинаем обработку 56 сочинений...
Ошибка при обработке essay_id 1: 'task_text'
Ошибка при обработке essay_id 2: 'task_text'
Ошибка при обработке essay_id 6: 'task_text'
Ошибка при обработке essay_id 7: 'task_text'
Ошибка при обработке essay_id 8: 'task_text'
Ошибка при обработке essay_id 9: 'task_text'
Ошибка при обработке essay_id 10: 'task_text'
Ошибка при обработке essay_id 11: 'task_text'
Ошибка при обработке essay_id 12: 'task_text'
Ошибка при обработке essay_id 13: 'task_text'
Ошибка при обработке essay_id 14: 'task_text'
Ошибка при обработке essay_id 15: 'task_text'
Ошибка при обработке essay_id 3: 'task_text'
Ошибка при обработке essay_id 4: 'task_text'
Ошибка при обработке essay_id 5: 'task_text'
Ошибка при обработке essay_id 41: 'task_text'
Ошибка при обработке essay_id 42: 'task_text'
Ошибка при обработке essay_id 43: 'task_text'
Ошибка при обработке essay_id 53: 'task_text'
Ошибка при обработке essay_id 55: 'task_text'
Ошибка при обработке essay_id 56: 'task_text'
Ошибка п

C:\Users\andre\AppData\Local\Temp\ipykernel_12860\1063364025.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=llm, prompt=prompt_type1, output_parser=parser)
